In [1]:
'''
#2 pytorch model
 깊은 신경망을 이용한 classification 문제 해결

'''

'\n#2 pytorch model\n 깊은 신경망을 이용한 classification 문제 해결\n\n'

In [ ]:
'''
#첫번째 레이어(convolutional layer)
합성곱(in_channel=1, out_channel=32, kernel_size=3, stride=1, padding=1) + 활성화함수(ReLU)
MaxPooling(kernel_size=2, stride=2)

#두번째 레이어(convolutional layer)
합성곱(in_channel=1, out_channel=64, kernel_size=3, stride=1, padding=1) + 활성화함수(ReLU)
MaxPooling(kernel_size=2, stride=2)

#세번째 레이어(convolutional layer)
합성곱(in_channel=1, out_channel=128, kernel_size=3, stride=1, padding=1) + 활성화함수(ReLU)
MaxPooling(kernel_size=2, stride=2, padding=1)

#네번째 레이어: 전결합층(FC layer)
특성맵을 펼치는 역할
FC(Fully connected) layer + 활성화 함수(ReLU)

#다섯번째 레이어: 전결합층(FC layer)
FC(Fully connected) layer + 활성화 함수(softmax)
마지막 노드 10개(0~9의 정답 가짐)

'''

In [3]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [4]:
#GPU가속화를 위한 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

#위의 고정 랜덤 시드는 GPU가 사용 가능할 경우에 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [5]:
#학습에 필요한 파라미터 설정
learning_rate = 0.001
training_epoch = 100
batch_size = 100

In [6]:
#torchvision데이터 로드로 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', train = True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='MNIST_data/', train = False, transform=transforms.ToTensor(), download=True)

In [7]:
#배치 크기를 지정하는 과정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size,shuffle=True,drop_last=True)
#60000개의 데이터가 batch_size 100의 크기로 진행되기 땜에 total batch는 600

In [10]:
#클래스 이용해 모델 설계
class CNN(torch.nn.Module):   #CNN구조를 사용한 모델을 선언하기 위한 클래스
    def __init__(self):   #초기화 함수, 동시에 모델의 구조를 선언하는 함수
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1), torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1), torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1), torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(self.fc1, torch.nn.ReLU(), torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self,x):   #순전파 전달을 위한 함수 forward선언
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        
        return out

In [11]:
#CNN 모델 생성
model = CNN().to(device)

In [12]:
#비용함수와 옵티마이저 선언
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
#총 배치의 수 확인
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))
#사용자가 지정한 배치의 수에 따라 총 배치의 수가 달라짐
#배치 횟수 변경, 에폭 횟수 변경, 옵티마이저 변경, 손실함수 변경 등 여러 방법으로 변경해보며 최적을 찾음

총 배치의 수 : 600


In [14]:
#학습 진행
for epoch in range(training_epoch):
    avg_cost = 0
    
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))


[Epoch:    1] cost = 0.190338954
[Epoch:    2] cost = 0.0496307574
[Epoch:    3] cost = 0.0360129066
[Epoch:    4] cost = 0.0277355313
[Epoch:    5] cost = 0.0227501206
[Epoch:    6] cost = 0.0186660942
[Epoch:    7] cost = 0.01595103
[Epoch:    8] cost = 0.0140637774
[Epoch:    9] cost = 0.0118345553
[Epoch:   10] cost = 0.0121402331
[Epoch:   11] cost = 0.0103960782
[Epoch:   12] cost = 0.00788633805
[Epoch:   13] cost = 0.0107634878
[Epoch:   14] cost = 0.00716589903
[Epoch:   15] cost = 0.00758429151
[Epoch:   16] cost = 0.00622492237
[Epoch:   17] cost = 0.00607722718
[Epoch:   18] cost = 0.00572458981
[Epoch:   19] cost = 0.00698140496
[Epoch:   20] cost = 0.00436912198
[Epoch:   21] cost = 0.00556471944
[Epoch:   22] cost = 0.00608149404
[Epoch:   23] cost = 0.00474241795
[Epoch:   24] cost = 0.00340674073
[Epoch:   25] cost = 0.00646336842
[Epoch:   26] cost = 0.00378193147
[Epoch:   27] cost = 0.00667339331
[Epoch:   28] cost = 0.00395984156
[Epoch:   29] cost = 0.00404394977


In [15]:
#모델 테스트
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\Users\03\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


Accuracy: 0.9495999813079834


C:\Users\03\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [21]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [22]:
#GPU가속화를 위한 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

#위의 고정 랜덤 시드는 GPU가 사용 가능할 경우에 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [23]:
#학습에 필요한 파라미터 설정
learning_rate = 0.001
training_epoch = 300
batch_size = 100

In [24]:
#torchvision데이터 로드로 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', train = True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='MNIST_data/', train = False, transform=transforms.ToTensor(), download=True)

In [25]:
#배치 크기를 지정하는 과정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size,shuffle=True,drop_last=True)
#60000개의 데이터가 batch_size 100의 크기로 진행되기 땜에 total batch는 600

In [26]:
#클래스 이용해 모델 설계
class CNN(torch.nn.Module):   #CNN구조를 사용한 모델을 선언하기 위한 클래스
    def __init__(self):   #초기화 함수, 동시에 모델의 구조를 선언하는 함수
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1), torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1), torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1), torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        self.layer4 = torch.nn.Sequential(torch.nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1), torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        self.layer5 = torch.nn.Sequential(torch.nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1), torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        
        self.fc1 = torch.nn.Linear(2 * 2 * 512, 1024, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer6 = torch.nn.Sequential(self.fc1, torch.nn.ReLU(), torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(1024, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self,x):   #순전파 전달을 위한 함수 forward선언
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.view(out.size(0),-1)
        out = self.layer6(out)
        out = self.fc2(out)
        
        return out

In [27]:
#CNN 모델 생성
model = CNN().to(device)

In [28]:
#비용함수와 옵티마이저 선언
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
#총 배치의 수 확인
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))
#사용자가 지정한 배치의 수에 따라 총 배치의 수가 달라짐
#배치 횟수 변경, 에폭 횟수 변경, 옵티마이저 변경, 손실함수 변경 등 여러 방법으로 변경해보며 최적을 찾음

총 배치의 수 : 600


In [30]:
#학습 진행
for epoch in range(training_epoch):
    avg_cost = 0
    
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))


[Epoch:    1] cost = 0.232411891
[Epoch:    2] cost = 0.0545868278
[Epoch:    3] cost = 0.0389369503
[Epoch:    4] cost = 0.0322857164
[Epoch:    5] cost = 0.0250493176
[Epoch:    6] cost = 0.0223017521
[Epoch:    7] cost = 0.023707781
[Epoch:    8] cost = 0.0154248774
[Epoch:    9] cost = 0.0162400901
[Epoch:   10] cost = 0.0146368574
[Epoch:   11] cost = 0.0153989382
[Epoch:   12] cost = 0.013651358
[Epoch:   13] cost = 0.0105976574
[Epoch:   14] cost = 0.0121625243
[Epoch:   15] cost = 0.00967344549
[Epoch:   16] cost = 0.0108475043
[Epoch:   17] cost = 0.00622705417
[Epoch:   18] cost = 0.0108079202
[Epoch:   19] cost = 0.00883001834
[Epoch:   20] cost = 0.00766856689
[Epoch:   21] cost = 0.0119372653
[Epoch:   22] cost = 0.00873011164
[Epoch:   23] cost = 0.00727194594
[Epoch:   24] cost = 0.0112380087
[Epoch:   25] cost = 0.00735833403
[Epoch:   26] cost = 0.00507608382
[Epoch:   27] cost = 0.00578441937
[Epoch:   28] cost = 0.0087206522
[Epoch:   29] cost = 0.0070328312
[Epoch: 

[Epoch:  238] cost = 0.0294149518
[Epoch:  239] cost = 0.0083475979
[Epoch:  240] cost = 0.00639538839
[Epoch:  241] cost = 0.00641675107
[Epoch:  242] cost = 0.00963312108
[Epoch:  243] cost = 0.025539482
[Epoch:  244] cost = 0.00679247826
[Epoch:  245] cost = 0.00243388861
[Epoch:  246] cost = 0.0397152156
[Epoch:  247] cost = 0.0103937229
[Epoch:  248] cost = 0.0047207321
[Epoch:  249] cost = 0.0133062359
[Epoch:  250] cost = 0.0146215791
[Epoch:  251] cost = 0.0143255945
[Epoch:  252] cost = 0.0102472976
[Epoch:  253] cost = 0.0171708222
[Epoch:  254] cost = 0.00616410375
[Epoch:  255] cost = 0.000897558348
[Epoch:  256] cost = 0.000545151939
[Epoch:  257] cost = 0.0165709294
[Epoch:  258] cost = 0.0445098504
[Epoch:  259] cost = 0.00713156443
[Epoch:  260] cost = 0.00952406786
[Epoch:  261] cost = 0.00291674817
[Epoch:  262] cost = 0.00272154831
[Epoch:  263] cost = 0.0248963106
[Epoch:  264] cost = 0.0178180374
[Epoch:  265] cost = 0.0114443852
[Epoch:  266] cost = 0.00508987531


In [31]:
#모델 테스트
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.807699978351593
